# Importing packages

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import collections as c
import os
import re
import numpy as np
import scipy.spatial.distance as sp
import math

ps = PorterStemmer() 
sno = SnowballStemmer("english")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\janep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\janep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Get list of file names

In [2]:
filenames = []
for fn in os.listdir("enron1\\ham"):
    if fn.endswith(".txt"):
        filenames.append(os.path.join("enron1\\ham", fn.replace("\\", "/")))
        
ham_num = len(filenames)

for fn in os.listdir("enron1\\spam"):
    if fn.endswith(".txt"):
        filenames.append(os.path.join("enron1\\spam", fn.replace("\\", "/")))
    
spam_num = len(filenames) - ham_num

In [3]:
len(filenames)

5172

# Stem email text

In [4]:
#flag to set initial df
first = True
stems_list = []
#iterate through each file
for fn in filenames:  
    #read file
    with open(fn, 'r', encoding="ISO-8859-1") as file:
        
        #remove numbers
        data = re.sub('[0-9]', '', file.read().replace('\n', ' '))
        
        #data = file.read().replace('\n', ' ')
        
        #tokenize + word-stem
        nltk_tokens = word_tokenize(data)
        stems = ""
        for w in nltk_tokens:
            stems += ps.stem(w) + " "
        
        stems_list.append(stems)


# Vectorize counts and get dictionary

In [5]:
#count vectorize
vect = CountVectorizer(input="content")
temp = vect.fit_transform(stems_list)
word_bag = vect.get_feature_names()



In [6]:
df = pd.DataFrame(temp.toarray(), columns=word_bag)
df

,aa,aaa,aabda,aabvmmq,aac,aachecar,aaer,aafco,aaiab,aaigrcrb,...,zynv,zyqtaqlt,zyrtec,zyyqywp,zzezrjok,zzn,zzo,zzocb,zzso,zzsyt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#dropping stop words
nonstop_attr = df.columns.copy()
for word in stopwords.words('english'):
    if word in df.columns:
        nonstop_attr = nonstop_attr.drop(word)

In [8]:
len(df.columns)-len(nonstop_attr)

119

# Divide data into ham, spam, test, and train

In [9]:
df_nonstop = df[nonstop_attr].copy()[nonstop_attr]

ham_data = df_nonstop[:ham_num]
spam_data = df_nonstop[ham_num:]
ham_data["SPAM_LABEL"] = 0
spam_data["SPAM_LABEL"] = 1

msk = np.random.rand(len(ham_data)) < 0.7
ham_train = ham_data[msk]
ham_test = ham_data[~msk]

msk = np.random.rand(len(spam_data)) < 0.7
spam_train = spam_data[msk]
spam_test = spam_data[~msk]

C:\Users\janep\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\janep\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
train_data = pd.concat([ham_train, spam_train])
test_data = pd.concat([ham_test, spam_test])

In [11]:
train_data

,aa,aaa,aabda,aabvmmq,aac,aachecar,aaer,aafco,aaiab,aaigrcrb,...,zyqtaqlt,zyrtec,zyyqywp,zzezrjok,zzn,zzo,zzocb,zzso,zzsyt,SPAM_LABEL
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# K Nearest Neighbors

This is old code which is pretty slow. We'll keep it for memory's sake.


In [12]:
#get data
# attr = train_data.columns.drop("SPAM_LABEL")
# train_data_knn = train_data.copy()
# test_data_knn = test_data.copy()
# test_data_knn["y_1"] = -1
# test_data_knn["y_10"] = -1
# test_data_knn["y_50"] = -1
# test_data_knn["y_200"] = -1
# test_data_knn["y_1000"] = -1

# for i in range(len(test_data_knn.index)):
#     temp = train_data_knn[attr].subtract(test_data_knn.iloc[i][attr])
#     train_data_knn["L2_NORM"] = temp.apply(np.linalg.norm, axis=1)
#     train_data_knn = train_data_knn.sort_values(by="L2_NORM", ascending=True)
#     test_data_knn["y_1"].iloc[i] = train_data_knn[:1]["SPAM_LABEL"].value_counts().idxmax()
#     test_data_knn["y_10"].iloc[i] = train_data_knn[:10]["two_year_recid"].value_counts().idxmax()
#     test_data_knn["y_50"].iloc[i] = train_data_knn[:50]["two_year_recid"].value_counts().idxmax()
#     test_data_knn["y_200"].iloc[i] = train_data_knn[:200]["two_year_recid"].value_counts().idxmax()
#     test_data_knn["y_1000"].iloc[i] = train_data_knn[:1000]["two_year_recid"].value_counts().idxmax()
    
    

Here's code that runs in 5-10 min. For some reason the accuracy is lower than above even though computation should be identical... could be variation from random seed splitting training and test.

In [13]:
#setting parameters
k = 5
attr = train_data.columns.drop("SPAM_LABEL")

#copying data
train_data_knn = train_data.copy()
test_data_knn = test_data.copy()

#get neighbors
print("Getting norms...")
norms = sp.cdist(test_data_knn, train_data_knn)
print("Getting neighbors...")
neighbors = np.apply_along_axis(np.argpartition, 0, norms, k)[:,:k]


#classifying
print("Classifying...")
test_data_knn["SPAM_LABEL_y"] = -1
for i in range(len(test_data_knn.index)):
    test_data_knn["SPAM_LABEL_y"] = train_data_knn.iloc[list(neighbors[i]), :]["SPAM_LABEL"].mode()[0]

Getting norms...
Getting neighbors...
Classifying...


In [14]:
len(test_data_knn[test_data_knn["SPAM_LABEL_y"] == test_data_knn["SPAM_LABEL"]].index)/len(test_data_knn.index)

0.7121794871794872

In [15]:
#get accuracy
len(test_data_knn[test_data_knn["SPAM_LABEL_y"] == test_data_knn["SPAM_LABEL"]].index)/len(test_data_knn.index)

0.7121794871794872

# Naive Bayes

We use add-one Laplace smoothing and log the probabilities to avoid zero-frequency problem.

In [17]:
def bayes_prob(row, freq, attr, total):
    p = 0
    for i in range(len(attr)):
        #using add-one laplacian smoothing
        p += math.log((freq[i][row[i+1]]+1)/total)
        
    return p


In [18]:
#get training data
train_data_b0 = train_data[train_data["SPAM_LABEL"]==0]
train_data_b1 = train_data[train_data["SPAM_LABEL"]==1]

#build frequency table
freq_0 = [c.Counter(train_data_b0[col]) for col in attr]
freq_1 = [c.Counter(train_data_b1[col]) for col in attr]

#get counts
counts = train_data["SPAM_LABEL"].value_counts()
total = counts.sum()

#get test data
test_data_bayes = test_data.copy()
test_data_bayes["SPAM_LABEL_y"] = -1


In [19]:
#classiying
for row in test_data.itertuples():
    p_0 = bayes_prob(row, freq_0, attr, counts[0]) + math.log(counts[0]/total)
    p_1 = bayes_prob(row, freq_1, attr, counts[1]) + math.log(counts[1]/total)
    
#     print(row[0])
#     print(p_0)
#     print(p_1)
    
    #give label
    if p_0 > p_1:
        test_data_bayes["SPAM_LABEL_y"].loc[row[0]] = 0
    else:
        test_data_bayes["SPAM_LABEL_y"].loc[row[0]] = 1

In [20]:
#accuracy
len(test_data_bayes[test_data_bayes["SPAM_LABEL_y"] == test_data_bayes["SPAM_LABEL"]].index)/len(test_data_bayes.index)

0.9115384615384615

In [26]:
b = {"qwer": 5}
b["f"]

KeyError: 'f'

# Decision tree

Lots of features = lots of time. We trim space by looking at topmost features with highest variance, then consider limited number of thresholds within each feature rather than every possible threshold. 

In [24]:
train_data_temp = train_data.copy()
test_data_d = test_data.copy()

msk = np.random.rand(len(train_data_temp)) < 0.9
train_data_d = train_data_temp[msk]
val = train_data_temp[~msk]
